In [ ]:
import numpy as np
import torch


dir = ["train", "validation", "test"]

#string to integer
strtoint_0 = lambda x:1. if x=="Bachelors" else (2. if x=="Masters" else 3.)    #Education
strtoint_2 = lambda x:1. if x=="Bangalore" else (2. if x=="Pune" else 3.)       #City
strtoint_4 = lambda x:1. if x=="Male" else 2.                                   #Gender
strtoint_5 = lambda x:1. if x=="Yes" else 2.                                    #ExperienceInCurrentDomain

convert_dict = {0:strtoint_0, 2:strtoint_2, 4:strtoint_4, 5:strtoint_5}

def load_data(dir):
    train = np.genfromtxt(f"dataset/{dir[0]}.csv", dtype=float, skip_header=1, encoding="UTF-8", converters=convert_dict, delimiter=",")
    validation = np.genfromtxt(f"dataset/{dir[1]}.csv", dtype=float, skip_header=1, encoding="UTF-8", converters=convert_dict, delimiter=",")
    test = np.genfromtxt(f"dataset/{dir[2]}.csv", dtype=float, skip_header=1, encoding="UTF-8", converters=convert_dict, delimiter=",")
    # label -1
    train[:,-1] -= 1.
    validation[:,-1] -= 1.

    return train, validation, test

train, validation, test = load_data(dir)
batch_size = 64

trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(validation, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm import trange
import torch.nn.functional as F
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# model = nn.Sequential(
#     nn.Linear(7, 24, dtype=torch.float32),
#     nn.ReLU(),
#     nn.Linear(24, 3, dtype=torch.float32)
# )

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.MLP1 = nn.Linear(7, 24, dtype=torch.float32)
        self.MLP2 = nn.Linear(24, 24, dtype=torch.float32)
        self.MLP3 = nn.Linear(24, 3, dtype=torch.float32)
    def forward(self, x):
        x = F.relu(self.MLP1(x))
        x = F.relu(self.MLP2(x))
        x = self.MLP3(x)
        return x

model = Net()
# model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

for epoch in trange(10):
    for i, data in enumerate(trainloader, 0):
        input = data[:,:-1].clone().type(torch.float32)
        label = data[:,-1].clone().long()
        optimizer.zero_grad()
        outputs = model(input)

        loss = loss_fn(outputs, label)
        loss.backward()
        optimizer.step()

        print(f'[{epoch + 1}, {i + 1:5d}] loss: {loss:.3f}')



In [ ]:
#validation set test
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        input = data[:,:-1].clone().type(torch.float32)
        label = data[:,-1].clone().long()
        outputs = model(input)
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

print(f'Accuracy : {100 * correct / total} %')